# Stream input data

In [ ]:
import warnings

import cartopy.io
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray as xr
from matplotlib.colors import LogNorm

import tams

warnings.filterwarnings("ignore", category=cartopy.io.DownloadWarning)

xr.set_options(display_expand_data=False)

In [ ]:
from textwrap import dedent

import earthaccess
from IPython.display import display, Markdown


def md(text):
    display(Markdown(dedent(text)))


HAVE_AUTH = False
try:
    _ = earthaccess.login()
except Exception:
    md("**Note:** Unable to log in to Earthdata. Data fetching will be skipped.")
else:
    HAVE_AUTH = True

## GPM precipitation and brightness temperature

With the help of [earthaccess](https://earthaccess.readthedocs.io/), we can stream these data without downloading the files to disk.
* {func}`tams.data.get_imerg`
* {func}`tams.data.get_mergir`

In [ ]:
%%time

if HAVE_AUTH:
    pr = tams.data.get_imerg("2024-06-01 02:30")["pr"]
else:
    pr = tams.data.load_example("docs-get-example-imerg")["pr"]

pr

In [ ]:
%%time

if HAVE_AUTH:
    tb = tams.data.get_mergir(pr.time.item())["tb"]
else:
    tb = tams.data.load_example("docs-get-example-mergir")["tb"]

tb

In [ ]:
%%time

fig = plt.figure(figsize=(12, 5), layout="constrained")
ax = fig.add_subplot(projection=ccrs.PlateCarree())

ax.coastlines(color="magenta")

cmap = plt.get_cmap("gist_gray_r")
cmap.set_bad("red")
tb.plot(x="lon", cmap=cmap, robust=True, ax=ax)
pr.plot(x="lon", norm=LogNorm(0.01, pr.quantile(0.99)), alpha=0.85, ax=ax);

The red dots in the brightness temperature field above represent missing data.
For most CE identification methods, this will influence results.
Since they are mostly scattered points, not large regions,
a reasonable way to fill in the missing data is a nearest-neighbor interpolation.

In [ ]:
%%time

kws = dict(method="nearest", fill_value="extrapolate", assume_sorted=True)
tb_ = tb.interpolate_na("lat", **kws).interpolate_na("lon", **kws)

print(f"{tb.isnull().sum().item() / tb.size:.3%} -> {tb_.isnull().sum().item() / tb_.size:.3%} null")

We zoom in to the system off the coast of Chile to demonstrate the impact.

In [ ]:
box = dict(lon=slice(-115, -80), lat=slice(-53, -20))

fig, [ax1, ax2] = plt.subplots(
    2, 1,
    figsize=(5, 7),
    sharex=True, sharey=True,
    subplot_kw=dict(projection=ccrs.Mercator()),
    layout="constrained",
)

tb.sel(box).plot(cmap=cmap, robust=True, ax=ax1)
ax1.set_title("")

tb_.sel(box).plot(cmap=cmap, robust=True, ax=ax2)
ax2.set_title("");